In [67]:
import yaml
import os
from pathlib import Path
from string import Template
from cognite.client import CogniteClient, global_config
from cognite.client.data_classes.filters import SpaceFilter
from cognite.client.data_classes.data_modeling.instances import InvolvedContainers

file_path = Path("cognite-sdk-config.yaml")

env_sub_template = Template(file_path.read_text())
file_env_parsed = env_sub_template.substitute(dict(os.environ))

cognite_config = yaml.safe_load(file_env_parsed)

global_config.apply_settings(cognite_config["global"])
client = CogniteClient.load(cognite_config["client"])


In [68]:
#list all containers from a specific space
space_name = "radix_space"
container_list = client.data_modeling.containers.list(space=space_name).to_pandas()
container_list


,space,external_id,properties,constraints,indexes,used_for,is_global,last_updated_time,created_time
0,radix_space,Location,"{'latitude': {'type': {'list': False, 'unit': ...",{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
1,radix_space,MetMast,"{'iecCompliant': {'type': {'list': False, 'typ...",{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
2,radix_space,RadixHorizontalActivity,{'RadixHorizontalActivityGUID': {'type': {'lis...,{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
3,radix_space,RadixHorizontalAsset,"{'location': {'type': {'list': False, 'type': ...",{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
4,radix_space,RadixHorizontalEquipment,{'RadixHorizontalEquipmentGUID': {'type': {'li...,{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
5,radix_space,RadixHorizontalTimeSeries,{'RadixHorizontalTimeSeriesGUID': {'type': {'l...,{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
6,radix_space,Substation,"{'voltageLevel': {'type': {'list': False, 'typ...",{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
7,radix_space,WindFarm,"{'capacityFactor': {'type': {'list': False, 't...",{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722
8,radix_space,WindTurbine,"{'activePower': {'type': {'list': False, 'type...",{},{},all,False,2025-03-28 00:03:27.722,2025-03-28 00:03:27.722


In [69]:
#delete all containers from a specific space
for c in container_list.external_id:
    print(c)
    client.data_modeling.containers.delete((space_name, c))

Location
MetMast
RadixHorizontalActivity
RadixHorizontalAsset
RadixHorizontalEquipment
RadixHorizontalTimeSeries
Substation
WindFarm
WindTurbine


In [70]:
#list all view from a specific space
view_list = client.data_modeling.views.list(space=space_name).to_pandas()
view_list

,space,external_id,implements,version,writable,used_for,is_global,properties,last_updated_time,created_time,name
0,radix_space,Location,"[{'space': 'cdf_cdm', 'external_id': 'CogniteD...",v1,True,all,False,"{'name': {'container': {'space': 'cdf_cdm', 'e...",2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,NaN
1,radix_space,MetMast,"[{'space': 'radix_space', 'external_id': 'Radi...",v1,True,node,False,{'object3D': {'container': {'space': 'cdf_cdm'...,2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,NaN
2,radix_space,RadixHorizontalActivity,"[{'space': 'cdf_cdm', 'external_id': 'CogniteA...",v1,True,node,False,"{'name': {'container': {'space': 'cdf_cdm', 'e...",2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,Activity
3,radix_space,RadixHorizontalAsset,"[{'space': 'cdf_cdm', 'external_id': 'CogniteA...",v1,True,node,False,{'object3D': {'container': {'space': 'cdf_cdm'...,2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,Asset
4,radix_space,RadixHorizontalEquipment,"[{'space': 'cdf_cdm', 'external_id': 'CogniteE...",v1,True,node,False,"{'name': {'container': {'space': 'cdf_cdm', 'e...",2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,Equipment
5,radix_space,RadixHorizontalTimeSeries,"[{'space': 'cdf_cdm', 'external_id': 'CogniteT...",v1,True,node,False,"{'name': {'container': {'space': 'cdf_cdm', 'e...",2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,Time series
6,radix_space,Substation,"[{'space': 'radix_space', 'external_id': 'Radi...",v1,True,node,False,{'object3D': {'container': {'space': 'cdf_cdm'...,2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,NaN
7,radix_space,WindFarm,"[{'space': 'radix_space', 'external_id': 'Radi...",v1,True,node,False,{'object3D': {'container': {'space': 'cdf_cdm'...,2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,NaN
8,radix_space,WindTurbine,"[{'space': 'radix_space', 'external_id': 'Radi...",v1,True,node,False,{'object3D': {'container': {'space': 'cdf_cdm'...,2025-03-28 00:03:29.422,2025-03-28 00:03:29.422,NaN


In [72]:
#delete all views from a specific space
for v in view_list.external_id:
    print(v)
    client.data_modeling.views.delete((space_name, v,"v1"))
#client.data_modeling.views.delete((space_name, "NeatOrgSequence","v1"))

Location
MetMast
RadixHorizontalActivity
RadixHorizontalAsset
RadixHorizontalEquipment
RadixHorizontalTimeSeries
Substation
WindFarm
WindTurbine


In [59]:
client.data_modeling.data_models.delete((space_name, "WindFarm","1"))

[DataModelId(space='wind_space', external_id='WindFarm', version='1')]

In [ ]:
dry_run = False
filter = SpaceFilter(space_name)
deleted_total = 0
for batch in client.data_modeling.instances(instance_type="node", filter=filter, chunk_size=1000):
    result = client.data_modeling.instances.inspect(
        nodes=batch.as_ids(),
        involved_containers=InvolvedContainers()
    )
    nodes_without_container_data = [
        (node.space, node.external_id) for node in result.nodes
        if not node.inspection_results.involved_containers
    ]
    if dry_run:
        for space, external_id in nodes_without_container_data:
            print(f"{space}:{external_id}")
    else:
        client.data_modeling.instances.delete(nodes=nodes_without_container_data)
        deleted_total += len(nodes_without_container_data)
        print(f"Deleted {len(nodes_without_container_data)} nodes without container data")

print(f"Deleted a total of {deleted_total} nodes")

CogniteAPIError: Attempted to delete a node which is used as a type | code: 400 | X-Request-ID: 86d07857-c6a9-9076-82c2-c9450c7ceffe | cluster: az-eastus-1
The API Failed to process some items.
Successful (2xx): []
Unknown (5xx): []
Failed (4xx): [NeatOrgAsset,NeatOrgTimeSeries,NeatOrgDataSet, ...]

In [ ]:
client.data_modeling.spaces.delete(space_name)